## Functions

In [1]:
#!pip3 install rich

import pprint

def print_(cursor, mode="pprint", tag=""):
    """
    Print collection.
    """
    if tag:
        print("%s=" % tag)
    if isinstance(cursor, pymongo.cursor.Cursor):
        # Serialize a cursor into a list.
        obj = list(cursor)
    else:
        obj = cursor
    if mode in ("json", "json_color"):
        import json

        parsed = json.loads(json.dumps(obj))
        if mode == "json":
            print(json.dumps(parsed, indent=2))
        else:
            import rich

            rich.print_json(json.dumps(parsed, indent=2))
    elif mode == "pprint":
        pprint.pprint(obj)
    else:
        raise ValueError(f"Invalid mode='{mode}'")

# Connect to DB

In [3]:
import pymongo
from bson.objectid import ObjectId

# Connect to MongoDB instance.
client = pymongo.MongoClient("localhost", 27017)

# Create a Mongo database.
db = client["book"]
print("db=", db)
print("type(db)=", type(db))

db= Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'book')
type(db)= <class 'pymongo.database.Database'>


In [5]:
# Show all the collections in the DB.
print("collections=", db.list_collection_names())

# Clean all the collections.
for collection_name in db.list_collection_names():
    print("Dropping collection %s" % collection_name)
    db[collection_name].drop()

# Show all the collections in the DB.
print("collections=", db.list_collection_names())

collections= []
collections= []


# `Towns` collection

## Insert

In [6]:
dict_ = {
    "name": "New York",
    "population": 22200000,
    "lastCensus": "2022-11-01",
    "famousFor": ["the MOMA", "food", "Derek Jeter"],
    "mayor": {"name": "Bill de Blasio", "party": "D"},
}
print_(dict_, tag="dict_")

# Inserting an object in a DB creates a DB.
val = db.towns.insert_one(dict_)
print("val=", val)
print("obj_id=", val.inserted_id)
print(dir(val))

dict_=
{'famousFor': ['the MOMA', 'food', 'Derek Jeter'],
 'lastCensus': '2022-11-01',
 'mayor': {'name': 'Bill de Blasio', 'party': 'D'},
 'name': 'New York',
 'population': 22200000}
val= InsertOneResult(ObjectId('65dfd3b9ab8388837bf6a610'), acknowledged=True)
obj_id= 65dfd3b9ab8388837bf6a610
['_InsertOneResult__inserted_id', '_WriteResult__acknowledged', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_raise_if_unacknowledged', 'acknowledged', 'inserted_id']


In [7]:
# Show all the collections.
db.list_collection_names()

['towns']

In [8]:
# Scan the collection.
for obj in db.towns.find():
    # _id is like the primary key.
    print_(obj, mode="pprint")

{'_id': ObjectId('65dfd3b9ab8388837bf6a610'),
 'famousFor': ['the MOMA', 'food', 'Derek Jeter'],
 'lastCensus': '2022-11-01',
 'mayor': {'name': 'Bill de Blasio', 'party': 'D'},
 'name': 'New York',
 'population': 22200000}


In [9]:
# Insert more data in the collection.
def insert_city(name, population, lastCensus, famousFor, mayor):
    db.towns.insert_one(
        {
            "name": name,
            "population": population,
            "lastCensus": lastCensus,
            "famousFor": famousFor,
            "mayor": mayor,
        }
    )


insert_city(
    "Punxsutawney",
    6200,
    "2016-01-31",
    ["Punxsutawney Phil"],
    # mayor.
    {"name": "Richard Alexander"},
)

insert_city(
    "Portland",
    582000,
    "2016-09-20",
    ["beer", "food", "Portlandia"],
    # mayor.
    {"name": "Ted Wheeler", "party": "D"},
)

# Note that `mayor` field doesn't have a strict schema.

In [10]:
# Print all the documents in db["towns"].
for obj in db.towns.find():
    print_(obj, mode="pprint")

{'_id': ObjectId('65dfd3b9ab8388837bf6a610'),
 'famousFor': ['the MOMA', 'food', 'Derek Jeter'],
 'lastCensus': '2022-11-01',
 'mayor': {'name': 'Bill de Blasio', 'party': 'D'},
 'name': 'New York',
 'population': 22200000}
{'_id': ObjectId('65dfd446ab8388837bf6a611'),
 'famousFor': ['Punxsutawney Phil'],
 'lastCensus': '2016-01-31',
 'mayor': {'name': 'Richard Alexander'},
 'name': 'Punxsutawney',
 'population': 6200}
{'_id': ObjectId('65dfd446ab8388837bf6a612'),
 'famousFor': ['beer', 'food', 'Portlandia'],
 'lastCensus': '2016-09-20',
 'mayor': {'name': 'Ted Wheeler', 'party': 'D'},
 'name': 'Portland',
 'population': 582000}


## Query

In [11]:
# Find by ObjectId.
# db.towns.find_one({"_id": ObjectId("6368352a657571ee34691dd9")})
db.towns.find_one({"_id": val.inserted_id})

{'_id': ObjectId('65dfd3b9ab8388837bf6a610'),
 'name': 'New York',
 'population': 22200000,
 'lastCensus': '2022-11-01',
 'famousFor': ['the MOMA', 'food', 'Derek Jeter'],
 'mayor': {'name': 'Bill de Blasio', 'party': 'D'}}

In [12]:
# Retrieve only the field `name` (i.e., projection).
object_id = ObjectId(str(val.inserted_id))
db.towns.find_one({"_id": object_id}, {"name": 1})

{'_id': ObjectId('65dfd3b9ab8388837bf6a610'), 'name': 'New York'}

In [13]:
# Retrieve all fields excluding `name`.
db.towns.find_one({"_id": ObjectId(str(val.inserted_id))}, {"name": 0})

{'_id': ObjectId('65dfd3b9ab8388837bf6a610'),
 'population': 22200000,
 'lastCensus': '2022-11-01',
 'famousFor': ['the MOMA', 'food', 'Derek Jeter'],
 'mayor': {'name': 'Bill de Blasio', 'party': 'D'}}

In [14]:
# Find all towns with name starting with P.
# This is going to do a table scan.
print_(db.towns.find({"name": {"$regex": r"^P"}}))

[{'_id': ObjectId('65dfd446ab8388837bf6a611'),
  'famousFor': ['Punxsutawney Phil'],
  'lastCensus': '2016-01-31',
  'mayor': {'name': 'Richard Alexander'},
  'name': 'Punxsutawney',
  'population': 6200},
 {'_id': ObjectId('65dfd446ab8388837bf6a612'),
  'famousFor': ['beer', 'food', 'Portlandia'],
  'lastCensus': '2016-09-20',
  'mayor': {'name': 'Ted Wheeler', 'party': 'D'},
  'name': 'Portland',
  'population': 582000}]


In [15]:
# Find all towns with name starting with P, but print only name.
print_(db.towns.find({"name": {"$regex": r"^P"}}, {"_id": 0, "name": 1}))

[{'name': 'Punxsutawney'}, {'name': 'Portland'}]


In [16]:
# Find all towns with name that begins with P and have population less than 100,000.
print_(db.towns.find({"name": {"$regex": r"^P"}, "population": {"$lt": 100000}}))

[{'_id': ObjectId('65dfd446ab8388837bf6a611'),
  'famousFor': ['Punxsutawney Phil'],
  'lastCensus': '2016-01-31',
  'mayor': {'name': 'Richard Alexander'},
  'name': 'Punxsutawney',
  'population': 6200}]


In [ ]:
# Projection.
print_(db.towns.find({"famousFor": "food"}, {"_id": 0, "name": 1, "famousFor": 1}))

# Note that the equality with an array is interpreted as "in".

In [ ]:
# Query for matching values.
print_(
    db.towns.find(
        {"famousFor": {"$all": ["food", "beer"]}}, {"_id": 0, "name": 1, "famousFor": 1}
    )
)

In [ ]:
# Query for lack of matching values.
print_(
    db.towns.find(
        {"famousFor": {"$nin": ["food", "beer"]}}, {"_id": 0, "name": 1, "famousFor": 1}
    )
)

In [ ]:
# Find results with nested search criteria, e.g., mayor.party = "D".
print_(db.towns.find({"mayor.party": "D"}))

## Updating

In [17]:
print_(db.towns.find())

[{'_id': ObjectId('65dfd3b9ab8388837bf6a610'),
  'famousFor': ['the MOMA', 'food', 'Derek Jeter'],
  'lastCensus': '2022-11-01',
  'mayor': {'name': 'Bill de Blasio', 'party': 'D'},
  'name': 'New York',
  'population': 22200000},
 {'_id': ObjectId('65dfd446ab8388837bf6a611'),
  'famousFor': ['Punxsutawney Phil'],
  'lastCensus': '2016-01-31',
  'mayor': {'name': 'Richard Alexander'},
  'name': 'Punxsutawney',
  'population': 6200},
 {'_id': ObjectId('65dfd446ab8388837bf6a612'),
  'famousFor': ['beer', 'food', 'Portlandia'],
  'lastCensus': '2016-09-20',
  'mayor': {'name': 'Ted Wheeler', 'party': 'D'},
  'name': 'Portland',
  'population': 582000}]


In [18]:
# Find the ID for a given document.
object_id_for_Portland = str(db.towns.find_one({"name": "Portland"})["_id"])
print("object_id_for_Portland=", object_id_for_Portland)

# Note that types matter, so searching for an _id as string doesn't work.
#print_(db.towns.find_one({"_id": object_id_for_Portland}))
print_(db.towns.find_one({"_id": ObjectId(object_id_for_Portland)}))

object_id_for_Portland= 65dfd446ab8388837bf6a612
{'_id': ObjectId('65dfd446ab8388837bf6a612'),
 'famousFor': ['beer', 'food', 'Portlandia'],
 'lastCensus': '2016-09-20',
 'mayor': {'name': 'Ted Wheeler', 'party': 'D'},
 'name': 'Portland',
 'population': 582000}


In [19]:
# There are multiple cities called Portland in US (e.g., in Oregon and in Maine).
# So add the state.
db.towns.update_one(
    {"_id": ObjectId(object_id_for_Portland)}, {"$set": {"state": "OR"}}
)

print_(db.towns.find({"_id": ObjectId(object_id_for_Portland)}))

# Note that we need to specify $set.
# Mongo thinks in terms of documents and not attributes. So if you specify:
# db.towns.update_one({"_id": ObjectId("63696c28657571ee34691de3")},
#                     {"state": "OR"})
# the entire document will be replaced with the document `{"state": "OR"}`

[{'_id': ObjectId('65dfd446ab8388837bf6a612'),
  'famousFor': ['beer', 'food', 'Portlandia'],
  'lastCensus': '2016-09-20',
  'mayor': {'name': 'Ted Wheeler', 'party': 'D'},
  'name': 'Portland',
  'population': 582000,
  'state': 'OR'}]


In [20]:
# Increment the population.
db.towns.update_one(
    {"_id": ObjectId(object_id_for_Portland)}, {"$inc": {"population": 1000}}
)
print_(db.towns.find({"_id": ObjectId(object_id_for_Portland)}))

[{'_id': ObjectId('65dfd446ab8388837bf6a612'),
  'famousFor': ['beer', 'food', 'Portlandia'],
  'lastCensus': '2016-09-20',
  'mayor': {'name': 'Ted Wheeler', 'party': 'D'},
  'name': 'Portland',
  'population': 583000,
  'state': 'OR'}]


# `countries` collection

## Insert

In [21]:
# Delete collection, if it exists.
db.countries.drop()

# Note that:
# 1) we define the _id directly
# 2) the schema is not strict
db.countries.insert_one({
    "_id": "us",
    "name": "United States",
    "exports": {
        "foods": [{
            "name": "bacon",
            "tasty": True
        }, {
            "name": "burgers"
        }]
    },
})

db.countries.insert_one({
    "_id": "ca",
    "name": "Canada",
    "exports": {
        "foods": [
            {
                "name": "bacon",
                "tasty": False
            },
            {
                "name": "syrup",
                "tasty": True
            },
        ]
    },
})

db.countries.insert_one({
    "_id": "mx",
    "name": "Mexico",
    "exports": {
        "foods": [{
            "name": "salsa",
            "tasty": True,
            "condiment": True
        }]
    },
})

assert db.countries.count_documents({}) == 3

In [22]:
for obj in db["countries"].find():
    print_(obj)

{'_id': 'us',
 'exports': {'foods': [{'name': 'bacon', 'tasty': True}, {'name': 'burgers'}]},
 'name': 'United States'}
{'_id': 'ca',
 'exports': {'foods': [{'name': 'bacon', 'tasty': False},
                       {'name': 'syrup', 'tasty': True}]},
 'name': 'Canada'}
{'_id': 'mx',
 'exports': {'foods': [{'condiment': True, 'name': 'salsa', 'tasty': True}]},
 'name': 'Mexico'}


## Query

In [23]:
# Find the country that exports tasty bacon.

# This doesn't return what we want, since we want the AND of the condition and not OR.
print_(
    db.countries.find(
        {
            "exports.foods.name": "bacon",
            "exports.foods.tasty": True,
        },
        {"_id": 0, "name": 1},
    )
)

[{'name': 'United States'}, {'name': 'Canada'}]


In [24]:
# Using $elemMatch.
print_(
    db.countries.find(
        {
            "exports.foods": {
                "$elemMatch": {
                    "name": "bacon",
                    "tasty": True,
                }
            }
        },
        {"_id": 0, "name": 1},
    )
)

[{'name': 'United States'}]


In [ ]:
# This performs an AND.
print_(db.countries.find({"_id": "mx", "name": "United States"}))

# This performs an OR.
print_(
    db.countries.find({"$or": [{"_id": "mx"}, {"name": "United States"}]}, {"_id": 1})
)

## References

In [25]:
object_id_for_Pun = ObjectId(str(db.towns.find_one({"name": "Punxsutawney"})["_id"]))
print("object_id_for_Pun=", object_id_for_Pun)

object_id_for_Pun= 65dfd446ab8388837bf6a611


In [26]:
# Mongo is not built to perform joins.
# It is useful to have documents reference each other.
db.towns.update_one(
    {"_id": object_id_for_Pun},
    {"$set": {"country": {"$ref": "countries", "$id": "us"}}},
)

print_(db.towns.find_one({"_id": object_id_for_Pun}))

{'_id': ObjectId('65dfd446ab8388837bf6a611'),
 'country': DBRef('countries', 'us'),
 'famousFor': ['Punxsutawney Phil'],
 'lastCensus': '2016-01-31',
 'mayor': {'name': 'Richard Alexander'},
 'name': 'Punxsutawney',
 'population': 6200}


In [27]:
var = db.towns.find_one({"_id": object_id_for_Pun})
print("var=", var)
print('var["country"]=', var["country"])
# Dereference.
print(var["country"].id)

var= {'_id': ObjectId('65dfd446ab8388837bf6a611'), 'name': 'Punxsutawney', 'population': 6200, 'lastCensus': '2016-01-31', 'famousFor': ['Punxsutawney Phil'], 'mayor': {'name': 'Richard Alexander'}, 'country': DBRef('countries', 'us')}
var["country"]= DBRef('countries', 'us')
us


## Delete

In [ ]:
# Find all contries where the bacon is not tasty.
bad_bacon = {
    "exports.foods": {
        "$elemMatch": {
            "name": "bacon",
            "tasty": False,
        }
    }
}
print_(db.countries.find(bad_bacon))

In [ ]:
print_(db.countries.find())

In [ ]:
print("count=", db.countries.count_documents({}))
db.countries.delete_many(bad_bacon)
print("count=", db.countries.count_documents({}))

## Query with code

# Indexing

In [ ]:
import random

random.seed(1)


def populatePhones(area, start, stop):
    for i in range(start, stop):
        country = 1 + random.randint(1, 8)
        num = int(country * 1e10 + area * 1e7 + i)
        # +4 800-5550000
        full_number = "+%s %s-%s" % (country, area, i)
        # print(num, full_number)
        # assert 0
        db.phones.insert_one(
            {
                "_id": num,
                "components": {
                    "country": country,
                    "area": area,
                    "number": i,
                },
                "display": full_number,
            }
        )


# Generate 100,000 phone numbers (it may take a while), between 1-800-555-0000 and 1-800-565-0000.
db.phones.drop()
populatePhones(800, 5550000, 5650000)

In [ ]:
print(db.phones.count_documents({}))

In [ ]:
print_(db.phones.find().limit(2))
print_(db.phones.find().limit(2))

In [ ]:
# Print information about the indices.
for collection in db.list_collection_names():
    print("# collection=", collection)
    print_(db[collection].index_information())

In [ ]:
print_(db.phones.find_one({"display": "+4 800-5550000"}))

In [ ]:
# db.phones.find({"display": "+4 800-5550000"}).explain()
db.phones.find({"display": "+4 800-5550000"}).explain()["executionStats"][
    "executionTimeMillis"
]

In [ ]:
# Create an indesx on `display`.
db.phones.create_index([("display", pymongo.ASCENDING)], unique=True, dropDups=True)

print_(db["phones"].index_information())

In [ ]:
# Show that the query now it's very fast.
print_(
    db.phones.find({"display": "+4 800-5550000"}).explain()["executionStats"][
        "executionTimeMillis"
    ]
)

# Aggregated queries.

In [ ]:
db.phones.count_documents({"components.number": {"$gt": 5599999}})

In [ ]:
db.phones.distinct('components.number', {"components.number": {"$gt": 5599999}})